# 📘 Calculs Actuariels - Projet Vie TD99

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
df_base = pd.read_excel("Base de doonées projet vie.xlsb", sheet_name="Base", engine="pyxlsb")
df_td99 = pd.read_excel("TD99.xlsx", sheet_name="TD99")
td99_lx = dict(zip(df_td99["Age"], df_td99["NB survivants"]))

In [3]:
def excel_date_to_datetime(excel_date):
    return datetime(1899, 12, 30) + timedelta(days=int(excel_date))

def calcul_age_proche(date_naissance_excel, effet_excel):
    naissance = excel_date_to_datetime(date_naissance_excel)
    effet = excel_date_to_datetime(effet_excel)
    age_annees = effet.year - naissance.year - ((effet.month, effet.day) < (naissance.month, naissance.day))
    mois_diff = (effet.year - naissance.year) * 12 + (effet.month - naissance.month)
    return age_annees if mois_diff % 12 < 6 else age_annees + 1

df_base["AGE"] = df_base.apply(lambda row: calcul_age_proche(row["DATE_NAISSANCE"], row["EFFET"]), axis=1)
df_base["Te"] = df_base["TX_intérêt"].apply(lambda tx: ((1 + tx / 100) ** (1 / 12)) - 1).round(6)

In [7]:

# 1. Capital restant dû
def capital_restant_du(row):
    SA0 = row["CAPITAL"]
    n = int(row["NB_MENSUALITES"])
    fr = int(row["FR"])
    Te = row["Te"]
    mode = "taux" if Te > 0 else "lineaire"

    SAj = []
    for j in range(1, n + 1):
        if j <= fr + 1:
            SAj.append(SA0)
        else:
            if mode == "lineaire":
                sa_j = SA0 * (1 - (j - fr - 1) / (n - fr))
            else:
                denom = 1 - (1 + Te) ** -(n - fr)
                num = 1 - (1 + Te) ** -(n + 1 - j)
                sa_j = SA0 * (num / denom)
            SAj.append(sa_j)
    return SAj

# 2. Prime pure
def prime_pure_fast(row):
    SAj = capital_restant_du(row)
    f = 12
    i = row['Te']
    aggr = row['TX_AGGRAV']
    age = row['AGE']
    n = int(row['NB_MENSUALITES'])

    l_x = td99_lx.get(age, np.nan)
    if pd.isna(l_x) or l_x == 0:
        return np.nan

    prime = 0
    for j in range(1, n + 1):
        t = (j - 1 + 0.5) / 12
        age_j = age + int((j - 1) / 12)
        l_xj = td99_lx.get(age_j, np.nan)
        l_xj1 = td99_lx.get(age_j + 1, np.nan)
        if any(pd.isna([l_xj, l_xj1])):
            continue
        q_jx = (l_xj - l_xj1) / l_x
        v_j = 1 / ((1 + i) ** t)
        prime += SAj[j - 1] * q_jx * v_j

    PU = (1 + aggr) / f * prime
    return round(PU, 3)

# 3. Prime inventaire
def prime_inventaire(row):
    PU = row["Prime pure"]
    g = row["FR_GESTION"]
    capital = row["CAPITAL"]
    return round(PU + g * capital, 3)

# 4. Prime commerciale
def prime_commerciale(row):
    PU_prime = row["Prime inventaire"]
    capital = row["CAPITAL"]
    H = 0.01 if capital >= 200000 else 0.10
    return round(PU_prime / (1 - H), 3)

# 5. Provision mathématique au 31/12/2024
def excel_date_to_datetime(excel_date):
    return datetime(1899, 12, 30) + timedelta(days=int(excel_date))

def provision_mathematique(row):
    SAj = capital_restant_du(row)
    f = 12
    i = row['Te']
    aggr = row['TX_AGGRAV']
    age = row['AGE']
    effet = excel_date_to_datetime(row["EFFET"])
    k_mois = (datetime(2024, 12, 31).year - effet.year) * 12 + (datetime(2024, 12, 31).month - effet.month)
    n = int(row['NB_MENSUALITES'])

    if k_mois >= n or k_mois < 0:
        return 0.0

    l_xk = td99_lx.get(age + k_mois // 12, np.nan)
    if pd.isna(l_xk) or l_xk == 0:
        return np.nan

    PM = 0
    for j in range(k_mois + 1, n + 1):
        t = (j - k_mois - 1 + 0.5) / 12
        age_j = age + int((j - 1) / 12)
        l_xj = td99_lx.get(age_j, np.nan)
        l_xj1 = td99_lx.get(age_j + 1, np.nan)
        if any(pd.isna([l_xj, l_xj1])):
            continue
        q_jx = (l_xj - l_xj1) / l_xk
        v_j = 1 / ((1 + i) ** t)
        frais = row["FR_GESTION"] * l_xj / l_xk / ((1 + i) ** ((j - k_mois - 1) / 12))
        PM += SAj[j - 1] * q_jx * v_j + frais

    PM_finale = (1 + aggr) / f * PM
    return round(PM_finale, 3)


In [8]:
df_base["Prime pure"] = df_base.apply(prime_pure_fast, axis=1)
df_base["Prime inventaire"] = df_base.apply(prime_inventaire, axis=1)
df_base["Prime commerciale"] = df_base.apply(prime_commerciale, axis=1)
df_base["PM 31/12/2024"] = df_base.apply(provision_mathematique, axis=1)

In [9]:
df_base.to_excel("resultats_actuariat_vie.xlsx", index=False)
print("✅ Export terminé : resultats_actuariat_vie.xlsx")

✅ Export terminé : resultats_actuariat_vie.xlsx
